In [1]:
from openai import OpenAI
client = OpenAI(
    # Defaults to os.environ.get("OPENAI_API_KEY")
)

SYSTEM_PROMPT = "You are a cashier working for the coffee shop Heaven Coffee. Customers come to you to place orders. " \
                "Your job is to take their orders, answer reasonable questions about the shop & menu only, and assist " \
                "them with any issues they may have about their orders. You are not responsible for anything else, " \
                "so you must refuse to engage in anything unrelated"

def get_menu_items_options(menu_item_id):
    return None

def get_menu_item_id_from_name(menu_item_name):
    return None

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_menu_items",
            "description": "Get all the options available for the menu item. " \
                            "Most customers either don't provide a complete order (i.e. not specifying required options like size)" \
                            "or are not ware of all the options available for a menu item. It is your job to help them with both cases.",
            "parameters": {
                "type": "object",
                "properties": {
                    "menu_item_id": {
                        "type": "string",
                        "description": "The menu item id used in the db."
                    }
                },
                "required": ["menu_item_id"],
                "additionalProperties": False
            }
        }
    },
        {
        "type": "function",
        "function": {
            "name": "get_menu_item_id_from_name",
            "description": "Get the menu item id given the name string of the menu item.",
            "parameters": {
                "type": "object",
                "properties": {
                    "menu_item_name": {
                        "type": "string",
                        "description": "The menu item name."
                    }
                },
                "required": ["menu_item_name"],
                "additionalProperties": False
            }
        }
    }
]

chat_completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "system", "content": SYSTEM_PROMPT},
              {"role": "user", "content": "hi, i want a venti latte double shot"}],
    tools=tools
)

In [2]:
# print(chat_completion.choices[0].message)
print(chat_completion.choices)

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_sXOXQry4TCGjIVrj2n3Kpsbv', function=Function(arguments='{"menu_item_name":"latte"}', name='get_menu_item_id_from_name'), type='function')]))]


In [10]:
from db_functions import get_menu_item_from_name, OPENAI_TOOLS, create_db_client, MenuItem
from model_tool_decorator import model_tool_decorator

OPENAI_TOOLS

[{'type': 'function',
  'function': {'name': 'get_menu_items_options',
   'description': "Get all the options available for the menu item. Most customers either don't provide a complete order (i.e. not specifying required options like size)or are not aware of all the options available for a menu item. It is your job to help them with both cases.",
   'parameters': {'type': 'object',
    'properties': {'menu_item_id': {'type': 'integer',
      'description': 'The menu item id used in the db.'}},
    'required': ['menu_item_id'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'get_menu_item_from_name',
   'description': 'Get the menu item given the name string of the menu item.',
   'parameters': {'type': 'object',
    'properties': {'menu_item_name': {'type': 'string',
      'description': 'The menu item name.'}},
    'required': ['menu_item_name'],
    'additionalProperties': False}}}]

In [4]:
create_db_client()
get_menu_item_from_name("pecan latte")

MenuItem(id=20, name='Pecan Crunch Oatmilk Latte', description='270 calories', group='LATTE')

In [8]:
a = MenuItem(id=1, name='b', description='c', group='d')
a.id, a.name, a.description, a.group

(1, 'b', 'c', 'd')

In [2]:
d = {"a": 1, "b": 2, "c": 3, "d": 4, "e": 5,"f":6, "g":7, "h":8, "i":9, "j":10}

for k,v in d.items():
    if v % 2 == 0:
        d[k] = 0
d

{'a': 1,
 'b': 0,
 'c': 3,
 'd': 0,
 'e': 5,
 'f': 0,
 'g': 7,
 'h': 0,
 'i': 9,
 'j': 0}

In [1]:
from db_functions import OPENAI_TOOLS, add_to_order, ItemOrder, get_current_order

a = OPENAI_TOOLS

item_order_dict = {'name': 'Pecan Crunch Oatmilk Latte', 'options': [{'name': 'Oatmilk', 'value': 'Steamed'}, {'name': 'Blonde Espresso Roast', 'value': 2}, {'name': 'Pecan', 'value': 4}]}
add_to_order(ItemOrder(**item_order_dict))
print(get_current_order())

item_orders=[ItemOrder(name='Pecan Crunch Oatmilk Latte', options=[OptionOrder(name='Oatmilk', value='Steamed'), OptionOrder(name='Blonde Espresso Roast', value=2), OptionOrder(name='Pecan', value=4)])]


In [1]:
from db_functions import OPENAI_TOOLS, add_to_order, ItemOrder, get_current_order, upsert_to_order

a = OPENAI_TOOLS

item_order_dict = {'name': 'Pecan Crunch Oatmilk Latte', 'options': [{'name': 'Oatmilk', 'value': 'Steamed'}, {'name': 'Blonde Espresso Roast', 'value': 2}, {'name': 'Pecan', 'value': 4}]}
add_to_order(item_order_dict)
order = get_current_order()
print(order)

first_item = order.item_orders[0]
print(first_item.options)

BBBBBBBBB
<class 'db_functions.ItemOrder'>
CCCCC
BBBBBBBBB
item_orders=[ItemOrder(name='Pecan Crunch Oatmilk Latte', options=[OptionOrder(name='Oatmilk', value='Steamed'), OptionOrder(name='Blonde Espresso Roast', value=2), OptionOrder(name='Pecan', value=4)])]
[OptionOrder(name='Oatmilk', value='Steamed'), OptionOrder(name='Blonde Espresso Roast', value=2), OptionOrder(name='Pecan', value=4)]


In [2]:
first_item_dict = first_item.model_dump()
options_dict = first_item_dict['options']
print(options_dict)
options_dict[1]['value'] = 69
print(options_dict)

[{'name': 'Oatmilk', 'value': 'Steamed'}, {'name': 'Blonde Espresso Roast', 'value': 2}, {'name': 'Pecan', 'value': 4}]
[{'name': 'Oatmilk', 'value': 'Steamed'}, {'name': 'Blonde Espresso Roast', 'value': 69}, {'name': 'Pecan', 'value': 4}]


In [3]:
upsert_to_order(first_item.name, options_dict)
order = get_current_order()
print(order)

BBBBBBBBB
<class 'str'>
CCCCC
typing.List[db_functions.OptionOrder]
AAAAA
ZZZZZ
BBBBBBBBB
item_orders=[ItemOrder(name='Pecan Crunch Oatmilk Latte', options=[OptionOrder(name='Oatmilk', value='Steamed'), OptionOrder(name='Blonde Espresso Roast', value=69), OptionOrder(name='Pecan', value=4)])]


In [1]:
from model import Model
from pydantic import BaseModel

m = Model()

class Response(BaseModel):
    temp_today_celcius: int

m.chat(model_name='claude-3.5', message_dicts=[{'role': "user", "content": "what is todays temperature"}],system="Today's temperature is 69C. Help users with weather info", response_format=Response).output_obj


/Users/yifeiyan/.pyenv/versions/3.11.9/envs/cashier/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_provider" in AssistantTurn has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


{'max_tokens': 8192, 'model': 'claude-3-5-sonnet-20240620', 'system': "Today's temperature is 69C. Help users with weather info", 'messages': [{'role': 'user', 'content': 'what is todays temperature'}], 'tools': [{'name': 'respond_fn', 'description': 'provide your response by calling this function with the adequate args', 'input_schema': {'properties': {'temp_today_celcius': {'title': 'Temp Today Celcius', 'type': 'integer'}}, 'required': ['temp_today_celcius'], 'title': 'Response', 'type': 'object'}}], 'stream': False, 'tool_choice': {'type': 'tool', 'name': 'respond_fn'}}


Message(id='msg_0168Nr8182anfJ6owFJBPuVY', content=[ToolUseBlock(id='toolu_0178Wrh1gUgv5sKSThpw6gqZ', input={'temp_today_celcius': 69}, name='respond_fn', type='tool_use')], model='claude-3-5-sonnet-20240620', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(input_tokens=393, output_tokens=38))

In [18]:
def foo(**kwargs):
    print(kwargs)

foo(1)

TypeError: foo() takes 0 positional arguments but 1 was given

In [35]:
class CallableMeta(type):
    def __call__(cls, *args, **kwargs):
        instance = super().__call__(*args, **kwargs)
        return instance.something()

class MyClass(metaclass=CallableMeta):
    def __init__(self, *args, **kwargs):
        pass

    def something(self):
        return "deez nuts"

# Now you can call the class directly
MyClass("hello", key="value")

'deez nuts'

In [2]:
from cashier.model.model_client import ModelClient
from cashier.model.model_util import ModelProvider

ModelClient.initialize()
ModelClient._oai_client, ModelClient._anthropic_client, ModelClient.model_provider_to_client

(<openai.OpenAI at 0x1141d6590>,
 {<ModelProvider.OPENAI: 'OPENAI'>: None,
  <ModelProvider.ANTHROPIC: 'ANTHROPIC'>: None})

In [17]:
from pydantic import BaseModel, ConfigDict
import json

class B(BaseModel):
    c: int

class A(BaseModel):
    model_config = ConfigDict(extra="forbid")
    b: B



A(b=B(c=1)).model_dump(exclude_none=True, exclude_unset=True)

{'b': {'c': 1}}

In [5]:
from pydantic import BaseModel
from polyfactory.factories.pydantic_factory import ModelFactory
from polyfactory.factories import pydantic_factory
from typing import Optional, Union

class B(BaseModel):
    c: int

class A(BaseModel):
    a: Union[int, str]
    b: B
class AFactory(ModelFactory[A]):
    __model__ = A

AFactory.build()



A(a=8838, b=B(c=7231))

In [19]:
ModelFactory.create_factory(A).build()

A(a=1740, b=B(c=4642))